# Creating VAE in keras to generate molecules

In [9]:
import tensorflow as tf
import numpy as np
from keras import backend as K
from keras.layers import Input, Dense, LSTM, Lambda, Reshape, TimeDistributed
from keras.models import Model
from keras import objectives
from keras.layers.core import RepeatVector
from keras.losses import MSE

with open('smiles.txt') as f:
    full_smiles_as_list = f.readlines()
    
SMILES_CHARS = set()
for smile in full_smiles_as_list:
    set_smile = set(smile)
    SMILES_CHARS = SMILES_CHARS.union(set_smile)

SMILES_CHARS = list(SMILES_CHARS)
SMILES_CHARS.insert(0,' ')

max_smiles_len = 100
num_smiles_chars =  len(SMILES_CHARS)

input_dim = (max_smiles_len, num_smiles_chars)
output_dim = (max_smiles_len, num_smiles_chars)


smi2index = dict((c, i) for i, c in enumerate(SMILES_CHARS))
index2smi = dict((i, c) for i, c in enumerate(SMILES_CHARS))

In [ ]:
latent_dim = 64
inter_dim = 512


In [2]:
from numpy.random import seed
seed(1)
tf.random.set_seed(2)

In [3]:
def smiles_to_onehot(smiles, max_len = max_smiles_len):
    onehot = np.zeros((max_len, len(SMILES_CHARS)))
    for i, c in enumerate(smiles):
        onehot[i, smi2index[c]] = 1
    return onehot


def smiles_decoder(onehot):
    smi = ''
    onehot = onehot.argmax( axis=-1 )
    for i in onehot:
        smi += index2smi[i]
    return smi

In [4]:
smiles_decoder(smiles_to_onehot(full_smiles_as_list[0]))

'C[C@@]1(C(=O)C=C(O1)C(=O)[O-])c2ccccc2\n                                                             '

## Training data

### shape(56, 100, 34)

In [ ]:
with open('smallsmiles.txt') as f:
    small_smiles_as_list = f.readlines()
    

In [12]:
X = [smiles_to_onehot(x) for x in small_smiles_as_list]
X = np.array(X)
X.shape

(56, 100, 57)

### shape(115936, 100, 34)

In [5]:
X_all = [smiles_to_onehot(x) for x in full_smiles_as_list]
X_all = np.array(X_all)
X_all.shape

(115936, 100, 35)

In [41]:
del full_smiles_as_list

In [36]:
X = X_all

In [6]:
X = X_all[:10000]
X.shape

(10000, 100, 35)

## Model of the encoder

In [7]:
def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

## Possible improvements <br>
- more lstm layers <br>
- lstm layer larger output dim <br>
- to avoid posteior collapse problem give less weight to KL loss

In [100]:
x_input = Input(shape=input_dim)
lstm = LSTM(inter_dim, return_sequences=True)(x_input)
lstm_inter = LSTM(inter_dim)(lstm)
z_mean = Dense(latent_dim)(lstm_inter)
z_var = Dense(latent_dim)(lstm_inter)

z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_var])
encoder = Model(x_input, z)

In [82]:
encoder.summary()

Model: "model_22"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 100, 34)      0                                            
__________________________________________________________________________________________________
lstm_16 (LSTM)                  (None, 100, 512)     1120256     input_4[0][0]                    
__________________________________________________________________________________________________
lstm_17 (LSTM)                  (None, 512)          2099200     lstm_16[0][0]                    
__________________________________________________________________________________________________
dense_10 (Dense)                (None, 64)           32832       lstm_17[0][0]                    
___________________________________________________________________________________________

## Model of the decoder

In [99]:
latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
#relud = Dense(latent_dim, name='latent_input', activation = 'relu')(latent_inputs)
repeated = RepeatVector(max_smiles_len)(latent_inputs)
hidden_lstm = LSTM(inter_dim, return_sequences=True)(repeated)
hidden_lstm = LSTM(inter_dim, return_sequences=True)(hidden_lstm)
x_2 = TimeDistributed(Dense(num_smiles_chars, activation='softmax'), name='decoded_mean')(hidden_lstm)

decoder = Model(latent_inputs, x_2)

In [83]:
decoder.summary()

Model: "model_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
z_sampling (InputLayer)      (None, 64)                0         
_________________________________________________________________
latent_input (Dense)         (None, 64)                4160      
_________________________________________________________________
repeat_vector_5 (RepeatVecto (None, 100, 64)           0         
_________________________________________________________________
lstm_21 (LSTM)               (None, 100, 512)          1181696   
_________________________________________________________________
lstm_22 (LSTM)               (None, 100, 512)          2099200   
_________________________________________________________________
decoded_mean (TimeDistribute (None, 100, 34)           17442     
Total params: 3,302,498
Trainable params: 3,302,498
Non-trainable params: 0
________________________________________________

### Loss functions

In [105]:
def calculate_loss(x, x_decoded):
    recon = max_smiles_len * num_smiles_chars * objectives.mse(x, x_decoded)
    
    kl = 0.5 * K.sum(K.exp(z_var) + K.square(z_mean) - 1. - z_var,axis = -1)
    kl *= 0.1
    return K.sum(recon + kl)

In [9]:
def z_loss(x, x_new):
    #xent_loss = objectives.mse(x, x_new)
    kl_loss = - 0.5 * K.mean(1 + z_var - K.square(z_mean) - K.exp(z_var))
    # loss = kl_loss + xent_loss
    return kl_loss

## Testing the decoder

In [104]:
decoder.compile(loss=MSE, optimizer='adam')

In [105]:
decoder.fit(Y, X, epochs=6, batch_size=10)

Epoch 1/6
56/56 [==============================] - 1s 12ms/step - loss: 0.0070
Epoch 2/6
56/56 [==============================] - 0s 4ms/step - loss: 0.0070
Epoch 3/6
56/56 [==============================] - 0s 4ms/step - loss: 0.0070
Epoch 4/6
56/56 [==============================] - 0s 4ms/step - loss: 0.0070
Epoch 5/6
56/56 [==============================] - 0s 4ms/step - loss: 0.0070
Epoch 6/6
56/56 [==============================] - 0s 4ms/step - loss: 0.0070


In [114]:
test_input = np.zeros((3,64))

In [115]:
result = decoder.predict(test_input)

In [116]:
result.shape

(3, 100, 57)

## Testing the encoder

In [23]:
encoder.compile(loss=z_loss, optimizer='adam')

In [24]:
encoder.fit(X, X, epochs=10, batch_size=10)

Epoch 1/10
56/56 [==============================] - 1s 11ms/step - loss: 12.5941
Epoch 2/10
56/56 [==============================] - 0s 4ms/step - loss: 0.0358
Epoch 3/10
56/56 [==============================] - 0s 4ms/step - loss: 0.0455
Epoch 4/10
56/56 [==============================] - 0s 4ms/step - loss: 0.0484
Epoch 5/10
56/56 [==============================] - 0s 4ms/step - loss: 0.0487
Epoch 6/10
56/56 [==============================] - 0s 4ms/step - loss: 0.0480
Epoch 7/10
56/56 [==============================] - 0s 4ms/step - loss: 0.0466
Epoch 8/10
56/56 [==============================] - 0s 4ms/step - loss: 0.0451
Epoch 9/10
56/56 [==============================] - 0s 4ms/step - loss: 0.0434
Epoch 10/10
56/56 [==============================] - 0s 4ms/step - loss: 0.0418


In [26]:
X1 = X[0:1]
X1 = np.reshape(X1, (1,100,57))

In [27]:
vec = encoder.predict(X1)
vec

array([[-8.1301773e-01, -2.6299402e-01, -9.8160648e-01, -1.2278748e+00,
        -1.1572304e+00, -1.5604430e+00, -1.3140545e+00, -7.6917696e-01,
        -2.4372327e+00,  1.0624061e+00, -3.0992565e-01,  8.3419043e-01,
         5.6062818e-01, -1.3172098e-01, -9.5717466e-01, -2.1324940e+00,
         5.0561869e-01,  2.7011657e-01,  5.0852150e-01, -1.7143270e+00,
         8.7156326e-01,  7.9136580e-01,  2.8402200e-01,  1.1757556e+00,
        -6.0712612e-01,  2.3732959e-01,  1.5342817e+00,  8.8119239e-01,
         1.3047813e+00, -7.7862233e-01, -1.4512120e-03,  1.3836908e-01,
        -1.2058165e+00, -1.8850985e+00, -7.6009864e-01, -1.7057220e+00,
        -1.0867665e+00, -1.8097383e+00,  1.2657901e+00, -2.4245261e-01,
         1.1391633e+00,  5.5728078e-01, -1.2521151e-01,  3.1110209e-01,
         1.3163338e+00,  2.4253933e+00,  1.1826134e+00,  1.9561082e+00,
         9.5373905e-01, -4.7003338e-01,  1.5187393e+00, -1.9416742e-01,
        -3.8858828e-01,  3.5764900e-01, -8.0885172e-01,  1.90862

##  Compile VAE

In [106]:
outputs = decoder(encoder(x_input))

In [107]:
vae = Model(x_input,outputs)

In [108]:
vae.compile(loss=calculate_loss, optimizer='adam')

In [109]:
vae.fit(X,X, epochs=1, batch_size=75, validation_split=0.2)

Train on 8000 samples, validate on 2000 samples
Epoch 1/1


InvalidArgumentError:  Incompatible shapes: [75,100] vs. [75]
	 [[node loss_7/model_19_loss/calculate_loss/add_1 (defined at C:\Users\ivani\Anaconda3\lib\site-packages\tensorflow_core\python\framework\ops.py:1751) ]] [Op:__inference_keras_scratch_graph_63918]

Function call stack:
keras_scratch_graph


In [18]:
import joblib
joblib.dump(vae,'vae.joblib')

['vae.joblib']

In [24]:
vae.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100, 57)           0         
_________________________________________________________________
model_4 (Model)              (None, 64)                3332224   
_________________________________________________________________
model_3 (Model)              (None, 100, 57)           3410856   
Total params: 6,743,080
Trainable params: 6,743,080
Non-trainable params: 0
_________________________________________________________________


In [62]:
X1 = X[-10:]
X1 = np.reshape(X1, (10,100,35))

In [63]:
predicted = vae.predict(X1)

In [64]:
for ind in np.arange(0,predicted.shape[0]):
    print(smiles_decoder(predicted[ind]))

cccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccc
cccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccc
cccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccc
cccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccc
cccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccc
cccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccc
cccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccc
cccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccc
cccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccc
ccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccc

In [74]:
smiles_decoder(X1[0])

'C[C@@]1(C(=O)C=C(O1)C(=O)[O-])c2ccccc2\nPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPP'